In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import datetime

In [2]:
customers = pd.read_csv('Customers.csv')
print("Customers Data Loaded: ", customers.shape)

products = pd.read_csv('Products.csv')
print("Products Data Loaded: ", products.shape)

transactions = pd.read_csv('Transactions.csv')
print("Transactions Data Loaded: ", transactions.shape)

Customers Data Loaded:  (200, 4)
Products Data Loaded:  (100, 4)
Transactions Data Loaded:  (1000, 7)


In [3]:
# Checking for missing values
print("Missing Values in Customers:", customers.isnull().sum())
print("Missing Values in Products:", products.isnull().sum())
print("Missing Values in Transactions:", transactions.isnull().sum())

# Dropping rows with missing values
customers.dropna(inplace=True)
products.dropna(inplace=True)
transactions.dropna(inplace=True)

Missing Values in Customers: CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64
Missing Values in Products: ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64
Missing Values in Transactions: TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [4]:
# Convert SignupDate and TransactionDate to datetime
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

# Ensure numerical columns are correctly typed
transactions['Quantity'] = transactions['Quantity'].astype(int)
transactions['TotalValue'] = transactions['TotalValue'].astype(float)
products['Price'] = products['Price'].astype(float)

In [7]:
import pandas as pd
import numpy as np


# 1. Merge Transactions with Products to get Category and Price
transactions_products = transactions.merge(products, on='ProductID', how='left')

# 2. Compute total spend per customer
customer_spend = transactions.groupby('CustomerID')['TotalValue'].sum().reset_index()
customer_spend.rename(columns={'TotalValue': 'TotalSpend'}, inplace=True)

# 3. Compute average transaction value per customer
customer_avg_trans = transactions.groupby('CustomerID')['TotalValue'].mean().reset_index()
customer_avg_trans.rename(columns={'TotalValue': 'AvgTransactionValue'}, inplace=True)

# 4. Compute number of transactions per customer
customer_num_trans = transactions.groupby('CustomerID')['TransactionID'].nunique().reset_index()
customer_num_trans.rename(columns={'TransactionID': 'NumTransactions'}, inplace=True)

# 5. Determine favorite category per customer
favorite_category = transactions_products.groupby(['CustomerID', 'Category'])['Quantity'].sum().reset_index()
favorite_category = favorite_category.sort_values(['CustomerID', 'Quantity'], ascending=[True, False])
favorite_category = favorite_category.drop_duplicates(subset=['CustomerID'], keep='first')
favorite_category = favorite_category[['CustomerID', 'Category']].rename(columns={'Category': 'FavoriteCategory'})

# 6. Merge all customer features
customer_features = customers.merge(customer_spend, on='CustomerID', how='left') \
                             .merge(customer_avg_trans, on='CustomerID', how='left') \
                             .merge(customer_num_trans, on='CustomerID', how='left') \
                             .merge(favorite_category, on='CustomerID', how='left')

# 7. Fill any missing values resulting from merges
customer_features['TotalSpend'] = customer_features['TotalSpend'].fillna(0)
customer_features['AvgTransactionValue'] = customer_features['AvgTransactionValue'].fillna(0)
customer_features['NumTransactions'] = customer_features['NumTransactions'].fillna(0)
customer_features['FavoriteCategory'] = customer_features['FavoriteCategory'].fillna('Unknown')

# 8. Feature: Tenure in days from SignupDate to latest TransactionDate
latest_date = transactions['TransactionDate'].max()
customer_features['TenureDays'] = (latest_date - customer_features['SignupDate']).dt.days

# 9. Distinct Categories Purchased
distinct_categories = transactions_products.groupby('CustomerID')['Category'].nunique().reset_index()
distinct_categories.rename(columns={'Category': 'DistinctCategories'}, inplace=True)

# 10. Recency: Days since last transaction
last_transaction = transactions.groupby('CustomerID')['TransactionDate'].max().reset_index()
last_transaction['Recency'] = (latest_date - last_transaction['TransactionDate']).dt.days
last_transaction = last_transaction[['CustomerID', 'Recency']]

# 11. Merge these features
customer_features = customer_features.merge(distinct_categories, on='CustomerID', how='left') \
                                     .merge(last_transaction, on='CustomerID', how='left')

# 12. Fill missing values
customer_features['DistinctCategories'] = customer_features['DistinctCategories'].fillna(0)

# 13. Fill missing Recency values
# If a customer has no transactions, Recency can be set to their full tenure
customer_features['Recency'] = customer_features['Recency'].fillna(customer_features['TenureDays'])

# 14. Now, drop SignupDate and CustomerName as they are not needed for similarity
customer_features = customer_features.drop(['CustomerName', 'SignupDate'], axis=1)

print(customer_features.head())
print(customer_features.columns)

  CustomerID         Region  TotalSpend  AvgTransactionValue  NumTransactions  \
0      C0001  South America     3354.52              670.904              5.0   
1      C0002           Asia     1862.74              465.685              4.0   
2      C0003  South America     2725.38              681.345              4.0   
3      C0004  South America     5354.88              669.360              8.0   
4      C0005           Asia     2034.24              678.080              3.0   

  FavoriteCategory  TenureDays  DistinctCategories  Recency  
0      Electronics         902                 3.0     55.0  
1       Home Decor        1049                 2.0     25.0  
2       Home Decor         296                 3.0    125.0  
3       Home Decor         811                 3.0      4.0  
4      Electronics         866                 2.0     54.0  
Index(['CustomerID', 'Region', 'TotalSpend', 'AvgTransactionValue',
       'NumTransactions', 'FavoriteCategory', 'TenureDays',
       'Disti

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Defining feature columns
numerical_features = ['TotalSpend', 'AvgTransactionValue', 'NumTransactions',
                      'TenureDays', 'DistinctCategories', 'Recency']
categorical_features = ['Region', 'FavoriteCategory']

# Defining ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Fitting and transforming the data
feature_matrix = preprocessor.fit_transform(customer_features)

print(f"Type of feature_matrix: {type(feature_matrix)}")

if hasattr(feature_matrix, "toarray"):
    feature_matrix = feature_matrix.toarray()
    print("Converted sparse matrix to dense array.")
else:
    print("feature_matrix is already a dense NumPy array.")

# Get CustomerIDs
customer_ids = customer_features['CustomerID'].values

Type of feature_matrix: <class 'numpy.ndarray'>
feature_matrix is already a dense NumPy array.


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Computing cosine similarity matrix
similarity_matrix = cosine_similarity(feature_matrix)

# Converting to DataFrame for easier handling
similarity_df = pd.DataFrame(similarity_matrix, index=customer_ids, columns=customer_ids)

In [11]:
# Sort CustomerIDs to get first 20 (assuming they are sorted alphanumerically)
sorted_customers = sorted(customer_ids)
first_20_customers = sorted_customers[:20]
print("First 20 Customers:", first_20_customers)

lookalike_dict = {}

for customer in first_20_customers:
    sim_scores = similarity_df.loc[customer].copy()
    sim_scores[customer] = -1

    # Get top 3 customers with highest similarity
    top_3 = sim_scores.sort_values(ascending=False).head(3)

    lookalike_dict[customer] = list(zip(top_3.index, top_3.values))

First 20 Customers: ['C0001', 'C0002', 'C0003', 'C0004', 'C0005', 'C0006', 'C0007', 'C0008', 'C0009', 'C0010', 'C0011', 'C0012', 'C0013', 'C0014', 'C0015', 'C0016', 'C0017', 'C0018', 'C0019', 'C0020']


In [12]:
lookalike_data = []

for cust_id, lookalikes in lookalike_dict.items():
    for lookalike_id, score in lookalikes:
        lookalike_data.append({
            'CustomerID': cust_id,
            'LookalikeID': lookalike_id,
            'SimilarityScore': score
        })

lookalike_df = pd.DataFrame(lookalike_data)

lookalike_pivot = lookalike_df.groupby('CustomerID').apply(
    lambda x: list(zip(x['LookalikeID'], x['SimilarityScore']))
).reset_index(name='Lookalikes')

lookalike_pivot.to_csv('Lookalike.csv', index=False)

print("Lookalike.csv generated successfully.")

Lookalike.csv generated successfully.


<ipython-input-12-84bc4af1174d>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lookalike_pivot = lookalike_df.groupby('CustomerID').apply(
